In [1]:
# Import the required libraries and dependencies
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

## Creating Song Vectors

In [2]:
track_feature_df = pd.read_csv(Path("track_features.csv"))
track_feature_num_df = pd.read_csv(Path("track_feature_num.csv"))


In [3]:
track_feature_df

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,track_uri,track_href,analysis_url,duration_ms,time_signature
0,0,0.582,0.5910,1,-9.751,0,0.0361,0.773000,0.764000,0.1240,0.5470,145.074,2scVgQKMFnsHgj5ThLlW0j,spotify:track:2scVgQKMFnsHgj5ThLlW0j,https://api.spotify.com/v1/tracks/2scVgQKMFnsH...,https://api.spotify.com/v1/audio-analysis/2scV...,235000,4
1,1,0.578,0.9300,7,-9.914,1,0.0469,0.000195,0.883000,0.1010,0.0979,138.004,4mdMLQVgaleASbJSpmhQkJ,spotify:track:4mdMLQVgaleASbJSpmhQkJ,https://api.spotify.com/v1/tracks/4mdMLQVgaleA...,https://api.spotify.com/v1/audio-analysis/4mdM...,395761,4
2,2,0.519,0.7320,10,-7.086,1,0.0457,0.104000,0.000004,0.2430,0.9510,190.075,0210qTsUZo7WghrDKEZ9d2,spotify:track:0210qTsUZo7WghrDKEZ9d2,https://api.spotify.com/v1/tracks/0210qTsUZo7W...,https://api.spotify.com/v1/audio-analysis/0210...,192993,4
3,3,0.660,0.8210,10,-4.785,0,0.0337,0.000678,0.000019,0.0792,0.6060,100.027,0XTSTgamPnRIg6hgpp81po,spotify:track:0XTSTgamPnRIg6hgpp81po,https://api.spotify.com/v1/tracks/0XTSTgamPnRI...,https://api.spotify.com/v1/audio-analysis/0XTS...,201880,4
4,4,0.642,0.4700,5,-8.884,1,0.0284,0.482000,0.443000,0.0956,0.3510,100.913,6d2kJRcdqR2D1O8IES2WNE,spotify:track:6d2kJRcdqR2D1O8IES2WNE,https://api.spotify.com/v1/tracks/6d2kJRcdqR2D...,https://api.spotify.com/v1/audio-analysis/6d2k...,275333,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2214987,2214987,0.165,0.0706,7,-23.789,1,0.0449,0.979000,0.212000,0.0795,0.0383,84.155,2LZzf74X9UmnrJH4zVFspM,spotify:track:2LZzf74X9UmnrJH4zVFspM,https://api.spotify.com/v1/tracks/2LZzf74X9Umn...,https://api.spotify.com/v1/audio-analysis/2LZz...,152987,3
2214988,2214988,0.447,0.6720,0,-6.996,0,0.0310,0.091700,0.956000,0.1170,0.0779,142.011,4u7yBiQUOdwafJDBEVMbhv,spotify:track:4u7yBiQUOdwafJDBEVMbhv,https://api.spotify.com/v1/tracks/4u7yBiQUOdwa...,https://api.spotify.com/v1/audio-analysis/4u7y...,323000,4
2214989,2214989,0.319,0.0192,5,-26.226,0,0.0417,0.989000,0.000064,0.0812,0.0542,91.493,3Po4vdsyyhbvg6y8E1eC2a,spotify:track:3Po4vdsyyhbvg6y8E1eC2a,https://api.spotify.com/v1/tracks/3Po4vdsyyhbv...,https://api.spotify.com/v1/audio-analysis/3Po4...,235987,4
2214990,2214990,0.157,0.0215,5,-26.131,1,0.0404,0.955000,0.923000,0.1010,0.0332,138.342,2Br6gpXQufCdpqAmOgLeeC,spotify:track:2Br6gpXQufCdpqAmOgLeeC,https://api.spotify.com/v1/tracks/2Br6gpXQufCd...,https://api.spotify.com/v1/audio-analysis/2Br6...,346760,4


In [4]:
track_feature_df = track_feature_df.drop(columns='Unnamed: 0')


In [5]:
track_feature_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,track_uri,track_href,analysis_url,duration_ms,time_signature
0,0.582,0.591,1,-9.751,0,0.0361,0.773000,0.764000,0.1240,0.5470,145.074,2scVgQKMFnsHgj5ThLlW0j,spotify:track:2scVgQKMFnsHgj5ThLlW0j,https://api.spotify.com/v1/tracks/2scVgQKMFnsH...,https://api.spotify.com/v1/audio-analysis/2scV...,235000,4
1,0.578,0.930,7,-9.914,1,0.0469,0.000195,0.883000,0.1010,0.0979,138.004,4mdMLQVgaleASbJSpmhQkJ,spotify:track:4mdMLQVgaleASbJSpmhQkJ,https://api.spotify.com/v1/tracks/4mdMLQVgaleA...,https://api.spotify.com/v1/audio-analysis/4mdM...,395761,4
2,0.519,0.732,10,-7.086,1,0.0457,0.104000,0.000004,0.2430,0.9510,190.075,0210qTsUZo7WghrDKEZ9d2,spotify:track:0210qTsUZo7WghrDKEZ9d2,https://api.spotify.com/v1/tracks/0210qTsUZo7W...,https://api.spotify.com/v1/audio-analysis/0210...,192993,4
3,0.660,0.821,10,-4.785,0,0.0337,0.000678,0.000019,0.0792,0.6060,100.027,0XTSTgamPnRIg6hgpp81po,spotify:track:0XTSTgamPnRIg6hgpp81po,https://api.spotify.com/v1/tracks/0XTSTgamPnRI...,https://api.spotify.com/v1/audio-analysis/0XTS...,201880,4
4,0.642,0.470,5,-8.884,1,0.0284,0.482000,0.443000,0.0956,0.3510,100.913,6d2kJRcdqR2D1O8IES2WNE,spotify:track:6d2kJRcdqR2D1O8IES2WNE,https://api.spotify.com/v1/tracks/6d2kJRcdqR2D...,https://api.spotify.com/v1/audio-analysis/6d2k...,275333,4


In [6]:
track_feature_df.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'id', 'track_uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'],
      dtype='object')

In [7]:
track_feature_num_df = track_feature_df[['track_uri','danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms','time_signature']]
# track_feature_num_df.index = track_feature_df['track_uri']

In [8]:
track_feature_num_df.head()

,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,spotify:track:2scVgQKMFnsHgj5ThLlW0j,0.582,0.591,1,-9.751,0,0.0361,0.773000,0.764000,0.1240,0.5470,145.074,235000,4
1,spotify:track:4mdMLQVgaleASbJSpmhQkJ,0.578,0.930,7,-9.914,1,0.0469,0.000195,0.883000,0.1010,0.0979,138.004,395761,4
2,spotify:track:0210qTsUZo7WghrDKEZ9d2,0.519,0.732,10,-7.086,1,0.0457,0.104000,0.000004,0.2430,0.9510,190.075,192993,4
3,spotify:track:0XTSTgamPnRIg6hgpp81po,0.660,0.821,10,-4.785,0,0.0337,0.000678,0.000019,0.0792,0.6060,100.027,201880,4
4,spotify:track:6d2kJRcdqR2D1O8IES2WNE,0.642,0.470,5,-8.884,1,0.0284,0.482000,0.443000,0.0956,0.3510,100.913,275333,4


## Creating Playlist Vector
* The playlist here refers to the playlist we want to continue

In [9]:
# Create playlist_feature_df
playlist_feature_df = pd.read_csv(Path("playlist.csv"))

In [10]:
# Check out the playlists in the database
playlist_feature_df

,pid,name,num_tracks,track_name,track_uri,duration_ms,artist_name,artist_uri,album_name,album_uri
0,0,Throwbacks,52,Lose Control (feat. Ciara & Fat Man Scoop),spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,226863,Missy Elliott,spotify:artist:2wIVse2owClT7go1WT98tk,The Cookbook,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K
1,0,Throwbacks,52,Toxic,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,198800,Britney Spears,spotify:artist:26dSoYclwsYLMAKD3tpOr4,In The Zone,spotify:album:0z7pVBGOD7HCIB7S8eLkLI
2,0,Throwbacks,52,Crazy In Love,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,235933,Beyoncé,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Dangerously In Love (Alben für die Ewigkeit),spotify:album:25hVFAxTlDvXbx2X2QkUkE
3,0,Throwbacks,52,Rock Your Body,spotify:track:1AWQoqb9bSvzTjaLralEkT,267266,Justin Timberlake,spotify:artist:31TPClRtHm23RisEBtV3X7,Justified,spotify:album:6QPkyl04rXwTGlGlcYaRoW
4,0,Throwbacks,52,It Wasn't Me,spotify:track:1lzr43nnXAijIGYnCT8M8H,227600,Shaggy,spotify:artist:5EvFsr3kj42KNv97ZEnqij,Hot Shot,spotify:album:6NmFmPX56pcLBOFMhIiKvF
...,...,...,...,...,...,...,...,...,...,...
7322474,197740,Boomtown,76,Grease - 2007 Remastered Version,spotify:track:5Vrczz39CvlD3OGCa6utoA,207306,Frankie Valli,spotify:artist:3CDKmzJu6uwEGnPLLZffpD,Frankie Valli...Is The Word,spotify:album:0o2oPAxKGui4tvrrNgDtkc
7322475,197740,Boomtown,76,Love Is The Answer - Single Version,spotify:track:7xKGghZ59EGUT0Ft9BRVMr,281293,England Dan,spotify:artist:4fcPqfHfCnNnKTFT8GwSei,Rhino Hi-Five: England Dan & John Ford Coley,spotify:album:40V7QExJNlpQZaxZf1JTF5
7322476,197740,Boomtown,76,Lonely Girl,spotify:track:3FiEa2pVN5MzEA2wrtKIsv,178146,Charles Kelley,spotify:artist:2Hbxyk7qJ22i8wFEqgM3vC,The Driver,spotify:album:6YXkbmsVABrzPBO6Vknj2P
7322477,197740,Boomtown,76,Golden Years - Single Version; 2002 Remastered...,spotify:track:6V8RUNP2Yy1dxw18rXRLu5,209600,David Bowie,spotify:artist:0oSGxfWSnnOXhD2fKuz2Gy,Golden Years [David Bowie vs. KCRW],spotify:album:2ObOh59P7oEmLSvqSUjXxc


In [11]:
# Merge the playlist_feature_df with track_feature_num_df to get track features for the playlist
playlist_tracks_merge = playlist_feature_df.merge(track_feature_num_df, on='track_uri', how='left')

In [12]:
# Check the merged playlist
playlist_tracks_merge.isnull().sum()

pid                      0
name                     0
num_tracks               0
track_name              23
track_uri                0
duration_ms_x            0
artist_name              2
artist_uri               0
album_name               3
album_uri                0
danceability        157777
energy              157777
key                 157777
loudness            157777
mode                157777
speechiness         157777
acousticness        157777
instrumentalness    157777
liveness            157777
valence             157777
tempo               157777
duration_ms_y       157777
time_signature      157777
dtype: int64

In [13]:
playlist_tracks_merge

,pid,name,num_tracks,track_name,track_uri,duration_ms_x,artist_name,artist_uri,album_name,album_uri,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms_y,time_signature
0,0,Throwbacks,52,Lose Control (feat. Ciara & Fat Man Scoop),spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,226863,Missy Elliott,spotify:artist:2wIVse2owClT7go1WT98tk,The Cookbook,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,...,-7.105,0.0,0.1210,0.03110,0.006970,0.0471,0.810,125.461,226864.0,4.0
1,0,Throwbacks,52,Toxic,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,198800,Britney Spears,spotify:artist:26dSoYclwsYLMAKD3tpOr4,In The Zone,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,...,-3.914,0.0,0.1140,0.02490,0.025000,0.2420,0.924,143.040,198800.0,4.0
2,0,Throwbacks,52,Crazy In Love,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,235933,Beyoncé,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Dangerously In Love (Alben für die Ewigkeit),spotify:album:25hVFAxTlDvXbx2X2QkUkE,...,-6.583,0.0,0.2100,0.00238,0.000000,0.0598,0.701,99.259,235933.0,4.0
3,0,Throwbacks,52,Rock Your Body,spotify:track:1AWQoqb9bSvzTjaLralEkT,267266,Justin Timberlake,spotify:artist:31TPClRtHm23RisEBtV3X7,Justified,spotify:album:6QPkyl04rXwTGlGlcYaRoW,...,-6.055,0.0,0.1410,0.20100,0.000234,0.0521,0.817,100.972,267267.0,4.0
4,0,Throwbacks,52,It Wasn't Me,spotify:track:1lzr43nnXAijIGYnCT8M8H,227600,Shaggy,spotify:artist:5EvFsr3kj42KNv97ZEnqij,Hot Shot,spotify:album:6NmFmPX56pcLBOFMhIiKvF,...,-4.596,1.0,0.0713,0.05610,0.000000,0.3130,0.654,94.759,227600.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7322474,197740,Boomtown,76,Grease - 2007 Remastered Version,spotify:track:5Vrczz39CvlD3OGCa6utoA,207306,Frankie Valli,spotify:artist:3CDKmzJu6uwEGnPLLZffpD,Frankie Valli...Is The Word,spotify:album:0o2oPAxKGui4tvrrNgDtkc,...,-8.450,1.0,0.0481,0.15600,0.000000,0.1010,0.872,109.374,207307.0,4.0
7322475,197740,Boomtown,76,Love Is The Answer - Single Version,spotify:track:7xKGghZ59EGUT0Ft9BRVMr,281293,England Dan,spotify:artist:4fcPqfHfCnNnKTFT8GwSei,Rhino Hi-Five: England Dan & John Ford Coley,spotify:album:40V7QExJNlpQZaxZf1JTF5,...,-11.974,1.0,0.0512,0.19900,0.000367,0.1820,0.538,203.105,281293.0,4.0
7322476,197740,Boomtown,76,Lonely Girl,spotify:track:3FiEa2pVN5MzEA2wrtKIsv,178146,Charles Kelley,spotify:artist:2Hbxyk7qJ22i8wFEqgM3vC,The Driver,spotify:album:6YXkbmsVABrzPBO6Vknj2P,...,-6.135,0.0,0.0268,0.08410,0.000000,0.0977,0.724,109.979,178147.0,4.0
7322477,197740,Boomtown,76,Golden Years - Single Version; 2002 Remastered...,spotify:track:6V8RUNP2Yy1dxw18rXRLu5,209600,David Bowie,spotify:artist:0oSGxfWSnnOXhD2fKuz2Gy,Golden Years [David Bowie vs. KCRW],spotify:album:2ObOh59P7oEmLSvqSUjXxc,...,-10.662,1.0,0.0514,0.03520,0.000021,0.0742,0.881,108.841,209600.0,4.0


In [14]:
playlist_tracks_merge.columns

Index(['pid', 'name', 'num_tracks', 'track_name', 'track_uri', 'duration_ms_x',
       'artist_name', 'artist_uri', 'album_name', 'album_uri', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms_y',
       'time_signature'],
      dtype='object')

In [15]:
# Create new dataframe `playlist_tracks_merge_features_df` with wanted features
playlist_tracks_merge_features_df = playlist_tracks_merge[['pid','name', 'track_uri', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms_y', 'time_signature']]
# Check out playlist_tracks_merge_features_df
playlist_tracks_merge_features_df

,pid,name,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms_y,time_signature
0,0,Throwbacks,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.904,0.813,4.0,-7.105,0.0,0.1210,0.03110,0.006970,0.0471,0.810,125.461,226864.0,4.0
1,0,Throwbacks,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,0.774,0.838,5.0,-3.914,0.0,0.1140,0.02490,0.025000,0.2420,0.924,143.040,198800.0,4.0
2,0,Throwbacks,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,0.664,0.758,2.0,-6.583,0.0,0.2100,0.00238,0.000000,0.0598,0.701,99.259,235933.0,4.0
3,0,Throwbacks,spotify:track:1AWQoqb9bSvzTjaLralEkT,0.892,0.714,4.0,-6.055,0.0,0.1410,0.20100,0.000234,0.0521,0.817,100.972,267267.0,4.0
4,0,Throwbacks,spotify:track:1lzr43nnXAijIGYnCT8M8H,0.853,0.606,0.0,-4.596,1.0,0.0713,0.05610,0.000000,0.3130,0.654,94.759,227600.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7322474,197740,Boomtown,spotify:track:5Vrczz39CvlD3OGCa6utoA,0.820,0.464,11.0,-8.450,1.0,0.0481,0.15600,0.000000,0.1010,0.872,109.374,207307.0,4.0
7322475,197740,Boomtown,spotify:track:7xKGghZ59EGUT0Ft9BRVMr,0.384,0.569,3.0,-11.974,1.0,0.0512,0.19900,0.000367,0.1820,0.538,203.105,281293.0,4.0
7322476,197740,Boomtown,spotify:track:3FiEa2pVN5MzEA2wrtKIsv,0.783,0.663,1.0,-6.135,0.0,0.0268,0.08410,0.000000,0.0977,0.724,109.979,178147.0,4.0
7322477,197740,Boomtown,spotify:track:6V8RUNP2Yy1dxw18rXRLu5,0.680,0.729,11.0,-10.662,1.0,0.0514,0.03520,0.000021,0.0742,0.881,108.841,209600.0,4.0


In [16]:
## Write playlist_tracks_merge_features_df to csv file
#playlist_tracks_merge_features_df.to_csv('playlist_tracks_merge_features.csv')

In [17]:
# Use the mean of track_features for playlist features
# Alternatively, we can use the weighted mean (to be decided)
playlist_mean_df = playlist_tracks_merge_features_df.drop(columns=['track_uri','name'],axis=1).groupby('pid').mean()

In [18]:
# Check out the aggregated dataframe. Each `pid` represents a playlist.
playlist_mean_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms_y,time_signature
pid,,,,,,,,,,,,,
0,0.661061,0.780918,4.775510,-4.830939,0.673469,0.104929,0.081336,0.000716,0.186316,0.644816,121.858980,222280.285714,4.000000
1,0.493655,0.693368,4.447368,-8.181184,0.552632,0.092413,0.166215,0.229575,0.179221,0.483737,125.618053,290857.578947,3.789474
2,0.674049,0.697951,4.934426,-4.788918,0.508197,0.096751,0.257482,0.000669,0.168790,0.569705,115.354410,220038.836066,4.000000
3,0.513131,0.616546,5.139344,-9.632697,0.713115,0.067923,0.280629,0.207753,0.185007,0.441988,125.172148,228786.040984,3.950820
4,0.588625,0.635069,3.437500,-7.857563,0.812500,0.041250,0.188235,0.086869,0.164806,0.491625,124.814375,254779.437500,3.937500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
197736,0.668053,0.702211,5.736842,-6.071316,0.473684,0.201174,0.176006,0.000023,0.171232,0.457263,138.468632,256062.789474,4.000000
197737,0.623577,0.704885,4.980769,-6.228442,0.596154,0.066998,0.177568,0.001247,0.200956,0.563596,121.035288,233133.788462,3.961538
197738,0.683849,0.674302,5.037736,-5.786547,0.528302,0.091553,0.130415,0.005843,0.158034,0.464226,115.063415,203873.792453,3.981132


In [19]:
playlist_scaler = StandardScaler()
playlist_scaler.fit(playlist_mean_df)
scaled_playlist = playlist_scaler.transform(playlist_mean_df)
scaled_playlist_df = pd.DataFrame(scaled_playlist, columns=playlist_mean_df.columns)
scaled_playlist_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms_y,time_signature
0,0.576023,1.086869,-0.587282,1.024549,0.093046,0.218276,-0.919914,-0.475680,-0.023617,1.217696,0.082085,-0.333571,0.582697
1,-1.161084,0.424830,-1.072493,-0.329405,-0.680409,-0.006573,-0.431592,1.297894,-0.187963,-0.044084,0.584717,1.499630,-1.521703
2,0.710794,0.459481,-0.352299,1.041531,-0.964826,0.071356,0.093487,-0.476041,-0.429572,0.629328,-0.787652,-0.393489,0.582697
3,-0.958990,-0.156091,-0.049294,-0.916013,0.346806,-0.446557,0.226654,1.128781,-0.053935,-0.371116,0.525094,-0.159659,0.091095
4,-0.175619,-0.016024,-2.565749,-0.198618,0.982948,-0.925756,-0.304905,0.191973,-0.521850,0.017706,0.477256,0.535193,-0.042047


In [20]:
# Choose one playlist as input playlist
input_playlist_feature_df = scaled_playlist_df[scaled_playlist_df.index==1]
input_playlist_feature_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms_y,time_signature
1,-1.161084,0.42483,-1.072493,-0.329405,-0.680409,-0.006573,-0.431592,1.297894,-0.187963,-0.044084,0.584717,1.49963,-1.521703


## Preparing for data for comparing similarity
* Compare aggregated input playlist features to song features that are not in the playlist `new_songs_feature`

In [21]:
# Get the new_playlist_info
input_playlist_info = playlist_feature_df[playlist_feature_df['pid']==1]
input_playlist_info.head()


,pid,name,num_tracks,track_name,track_uri,duration_ms,artist_name,artist_uri,album_name,album_uri
52,1,Awesome Playlist,39,Eye of the Tiger,spotify:track:2HHtWyy5CgaQbC7XSoOb0e,243773,Survivor,spotify:artist:26bcq2nyj5GB7uRr558iQg,Eye Of The Tiger,spotify:album:4PT9VulQaQP6XR1xBI2x1W
53,1,Awesome Playlist,39,Libera Me From Hell (Tengen Toppa Gurren Lagann),spotify:track:1MYYt7h6amcrauCOoso3Gx,70294,Daniel Tidwell,spotify:artist:7zdmbPudNX4SQJXnYIuCTC,Versus Hollywood,spotify:album:3q8vR3PFV8kG1m1Iv8DpKq
54,1,Awesome Playlist,39,Pokémon Theme,spotify:track:3x2mJ2bjCIU70NrH49CtYR,65306,Daniel Tidwell,spotify:artist:7zdmbPudNX4SQJXnYIuCTC,Versus Hollywood,spotify:album:3q8vR3PFV8kG1m1Iv8DpKq
55,1,Awesome Playlist,39,Concerning Hobbits (The Lord of the Rings),spotify:track:1Pm3fq1SC6lUlNVBGZi3Em,108532,Daniel Tidwell,spotify:artist:7zdmbPudNX4SQJXnYIuCTC,Versus Hollywood,spotify:album:3q8vR3PFV8kG1m1Iv8DpKq
56,1,Awesome Playlist,39,The Blood of Cuchulainn (The Boondock Saints),spotify:track:1NXTEkIeRL59NK61QuhYUl,214268,Daniel Tidwell,spotify:artist:7zdmbPudNX4SQJXnYIuCTC,Versus Hollywood,spotify:album:3q8vR3PFV8kG1m1Iv8DpKq


In [22]:
# Get the `track_uri` for the songs in the input playlist
input_playlist_tracks = input_playlist_info['track_uri']

# Create new_songs_feature_df with songs not in the current playlist
new_songs_feature_df = track_feature_num_df[~track_feature_num_df['track_uri'].isin(input_playlist_tracks.values)]
new_songs_feature_df

,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,spotify:track:2scVgQKMFnsHgj5ThLlW0j,0.582,0.5910,1,-9.751,0,0.0361,0.773000,0.764000,0.1240,0.5470,145.074,235000,4
1,spotify:track:4mdMLQVgaleASbJSpmhQkJ,0.578,0.9300,7,-9.914,1,0.0469,0.000195,0.883000,0.1010,0.0979,138.004,395761,4
2,spotify:track:0210qTsUZo7WghrDKEZ9d2,0.519,0.7320,10,-7.086,1,0.0457,0.104000,0.000004,0.2430,0.9510,190.075,192993,4
3,spotify:track:0XTSTgamPnRIg6hgpp81po,0.660,0.8210,10,-4.785,0,0.0337,0.000678,0.000019,0.0792,0.6060,100.027,201880,4
4,spotify:track:6d2kJRcdqR2D1O8IES2WNE,0.642,0.4700,5,-8.884,1,0.0284,0.482000,0.443000,0.0956,0.3510,100.913,275333,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2214987,spotify:track:2LZzf74X9UmnrJH4zVFspM,0.165,0.0706,7,-23.789,1,0.0449,0.979000,0.212000,0.0795,0.0383,84.155,152987,3
2214988,spotify:track:4u7yBiQUOdwafJDBEVMbhv,0.447,0.6720,0,-6.996,0,0.0310,0.091700,0.956000,0.1170,0.0779,142.011,323000,4
2214989,spotify:track:3Po4vdsyyhbvg6y8E1eC2a,0.319,0.0192,5,-26.226,0,0.0417,0.989000,0.000064,0.0812,0.0542,91.493,235987,4
2214990,spotify:track:2Br6gpXQufCdpqAmOgLeeC,0.157,0.0215,5,-26.131,1,0.0404,0.955000,0.923000,0.1010,0.0332,138.342,346760,4


## Scaling

In [23]:
# Scale the data
track_scaler = StandardScaler()
track_scaler.fit(new_songs_feature_df.drop('track_uri',axis=1))

StandardScaler()

In [24]:
scaled_new_songs = track_scaler.transform(new_songs_feature_df.drop('track_uri',axis=1))
scaled_new_songs_df = pd.DataFrame(scaled_new_songs, columns=new_songs_feature_df.drop('track_uri',axis=1).columns)
scaled_new_songs_df['track_uri']=new_songs_feature_df['track_uri']
scaled_new_songs_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,track_uri
0,0.171142,0.022975,-1.197338,-0.016069,-1.377813,-0.461245,1.180264,1.554565,-0.448412,0.264096,0.838357,-0.079583,0.254496,spotify:track:2scVgQKMFnsHgj5ThLlW0j
1,0.149465,1.297306,0.488582,-0.045038,0.725788,-0.367478,-1.000368,1.895143,-0.569574,-1.399382,0.602048,0.948488,0.254496,spotify:track:4mdMLQVgaleASbJSpmhQkJ
2,-0.170263,0.553006,1.331542,0.457557,0.725788,-0.377897,-0.707461,-0.631992,0.178471,1.760522,2.342479,-0.348219,0.254496,spotify:track:0210qTsUZo7WghrDKEZ9d2
3,0.593833,0.887565,1.331542,0.866493,-1.377813,-0.482082,-0.999005,-0.631947,-0.684415,0.482633,-0.667303,-0.291387,0.254496,spotify:track:0XTSTgamPnRIg6hgpp81po
4,0.496289,-0.431874,-0.073391,0.138015,0.725788,-0.528098,0.359146,0.635863,-0.598021,-0.461893,-0.637689,0.178347,0.254496,spotify:track:6d2kJRcdqR2D1O8IES2WNE


In [25]:
input_playlist_feature_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms_y,time_signature
1,-1.161084,0.42483,-1.072493,-0.329405,-0.680409,-0.006573,-0.431592,1.297894,-0.187963,-0.044084,0.584717,1.49963,-1.521703


## Calculating Similarity

In [26]:
# Using cosine similarity
new_songs_feature_df['similarity'] = cosine_similarity(scaled_new_songs_df.drop('track_uri',axis=1).values, input_playlist_feature_df.values)
new_songs_df_top_100 = new_songs_feature_df.sort_values('similarity', ascending=False).head(100)
new_songs_df_top_100

/var/folders/jn/000q98496_9_y7jg5hj5hvw40000gn/T/ipykernel_16592/4104172394.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_songs_feature_df['similarity'] = cosine_similarity(scaled_new_songs_df.drop('track_uri',axis=1).values, input_playlist_feature_df.values)


,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,similarity
1228774,spotify:track:0ayPBN8g7XNBJffY2nanX8,0.311,0.753,0,-9.430,0,0.0603,0.14800,0.927,0.2550,0.2620,121.100,517253,3,0.941226
1468733,spotify:track:3U8SgyR50gegCmCEMYd1oy,0.103,0.844,0,-11.804,0,0.1070,0.00275,0.742,0.1110,0.2020,158.375,488243,3,0.936896
1157251,spotify:track:2g1Z4SzOt6tQAk7Wem4BHP,0.421,0.679,0,-8.758,0,0.0358,0.18100,0.861,0.1000,0.3210,140.551,452859,3,0.929308
1608210,spotify:track:4sI1G2Uzef7YhCPrVjkJfg,0.174,0.741,4,-7.354,0,0.0591,0.10100,0.896,0.2020,0.3770,152.893,566720,3,0.929215
170948,spotify:track:4lyEOEYubluCAza53ojPbj,0.296,0.567,1,-9.438,0,0.0327,0.30800,0.793,0.0883,0.5010,137.047,412133,3,0.929134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1257035,spotify:track:2gRZp4uk5JzR7rVv4CJLZw,0.242,0.900,2,-4.283,0,0.1170,0.00454,0.881,0.0810,0.0398,124.113,693387,3,0.866314
1844147,spotify:track:4AGesFtGV2tuoQpeUdAORn,0.333,0.468,2,-11.347,0,0.0494,0.54500,0.656,0.1570,0.2300,132.487,466613,3,0.865968
190370,spotify:track:7fHPFXqZsGnZkg45zCcgWn,0.419,0.556,3,-12.183,0,0.0376,0.35200,0.778,0.0958,0.5460,136.407,347587,3,0.865210
1539371,spotify:track:0GYdS8z87D1Sn6IYB2f9Bz,0.351,0.380,0,-11.390,0,0.0337,0.19700,0.939,0.1340,0.1750,127.235,431213,3,0.864644


In [27]:
# Request the audio of the song and image from Spotipy